In [15]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from PIL import Image

In [53]:
# Location of repo's / files
label_file = './labels/labels_fc.csv'
dataset = '../Data/Development' 

# Read in data, and print example's 
df = pd.read_csv(label_file)
df.head()

,file,image_width,image_height,x_min,x_max,y_min,y_max,class_text,class_label,run_id,object_type,start_distance_from_car,angle,speed,offset_from_road_center,scenario_type
0,female_casual\uv662fepTPbTm36bxgyTn\cam000001.png,752,480,NaN,NaN,NaN,NaN,background,0,uv662fepTPbTm36bxgyTn,female_casual,10,30.0,1,NaN,left
1,female_casual\uv662fepTPbTm36bxgyTn\cam000002.png,752,480,NaN,NaN,NaN,NaN,background,0,uv662fepTPbTm36bxgyTn,female_casual,10,30.0,1,NaN,left
2,female_casual\uv662fepTPbTm36bxgyTn\cam000003.png,752,480,NaN,NaN,NaN,NaN,background,0,uv662fepTPbTm36bxgyTn,female_casual,10,30.0,1,NaN,left
3,female_casual\uv662fepTPbTm36bxgyTn\cam000004.png,752,480,NaN,NaN,NaN,NaN,background,0,uv662fepTPbTm36bxgyTn,female_casual,10,30.0,1,NaN,left
4,female_casual\uv662fepTPbTm36bxgyTn\cam000005.png,752,480,NaN,NaN,NaN,NaN,background,0,uv662fepTPbTm36bxgyTn,female_casual,10,30.0,1,NaN,left


In [3]:
import tqdm.notebook as tq

In [5]:
if not os.path.isdir('vae_dataset'): 
    os.mkdir('vae_dataset')

In [52]:
def find_dimension(val):
    x = 64
    while (val // x > 0) and x < 256: x*=2
    return x

In [17]:
import time 
for index, row in tq.tqdm(df.iterrows(), total=df.shape[0]):
    parts = row['file'].replace('\\','/').split('/')
    path = 'vae_dataset'
    for part in parts:
        path = os.path.join(path, part)
        if not os.path.isdir(path) and not path.endswith('.png'):
            os.mkdir(path)
    
        # Extract BB information
    bounding_box = row.loc[(['x_min','x_max','y_min','y_max'])]
    
    # Check if they're NaN --> Take next if true. 
    if bounding_box.isnull().values.any(): 
        file_object.close()
        continue 
    else: 
        # Computing Label position 
        xmin, xmax, ymin, ymax = bounding_box.values
        x_centre = ((xmin+xmax)//2) 
        y_centre = ((ymin+ymax)//2) 
        width = find_dimension(xmax-xmin) 
        height = find_dimension(ymax-ymin) 
    
        # Loading image 
        im = Image.open(os.path.join(dataset, row['file']))
        
        # Setting the points for cropped image
        left = x_centre - width
        top = y_centre - height
        right = x_centre + width
        bottom = y_centre + height 
        
        # Save image 
        im1 = im.crop((left, top, right, bottom))
        im1.save(os.path.join(path, row['file']))
        
        break

  0%|          | 0/69095 [00:00<?, ?it/s]